In [2]:
import ee

# Authenticate with Earth Engine using your credentials
ee.Authenticate()

# Initialize Earth Engine
ee.Initialize()


ModuleNotFoundError: No module named 'ee'

In [2]:
import ee
import geemap

# Create an interactive map centered on Vijayawada
#m = geemap.Map(center=(16.499, 80.638), zoom=14)

# Create an interactive map centered on Hyderabad
m = geemap.Map(center=(17.385044, 78.486671), zoom=10.5)

# Display the map
m

Map(center=[17.385044, 78.486671], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…

In [3]:
# Define the region of interest (ROI) by drawing a rectangle on the map
roi = m.draw_last_feature.geometry()

# Get the drawn rectangle geometry
rectangle_geometry = roi

# Calculate the area of the rectangle in square meters
area_sq_meters = rectangle_geometry.area().getInfo()

# Convert area to square kilometers
area_sq_km = area_sq_meters / (1000*1000)  # 1 square kilometer = 1,000,000 square meters

print("Area of the Rectangle in Square Meters:", f"{area_sq_meters:.2f} square meters")
print("Area of the Rectangle in Square Kilometers:", f"{area_sq_km:.2f} square kilometers")

Area of the Rectangle in Square Meters: 18926504.48 square meters
Area of the Rectangle in Square Kilometers: 18.93 square kilometers


In [4]:
# Define the region of interest (ROI) by drawing a rectangle on the map 'm'
roi = m.draw_last_feature.geometry()

# Create a new map 'm3' centered on the ROI
m3 = geemap.Map(center=(16.499, 80.638), zoom=14)
m3.centerObject(roi, 14)

# Get Landsat imagery
image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterDate('2020-01-01', '2020-12-31') \
    .filterBounds(roi) \
    .median()

# Calculate NDWI, NDBI, and NDVI
ndwi = image.normalizedDifference(['B5', 'B3'])
ndbi = image.normalizedDifference(['B5', 'B6'])
ndvi = image.normalizedDifference(['B5', 'B4'])

# Create masks based on threshold values
water_mask = ndwi.gt(0.2)  # Adjust threshold as needed
non_vegetation_mask = ndbi.gt(0.2)  # Adjust threshold as needed
vegetation_mask = ndvi.gt(0.2)  # Adjust threshold as needed

# Combine the masks into a single image
result_image = water_mask.add(non_vegetation_mask.multiply(2)).add(vegetation_mask.multiply(3))

# Set visualization parameters
result_vis_params = {
    'min': 1,
    'max': 3,
    'palette': ['white', 'blue', 'green']
}

# Display the result image on map 'm3'
m3.addLayer(result_image, result_vis_params, 'Land Cover')

# Add the ROI (rectangle) with a red outline to map 'm3' on top
m3.addLayer(roi, {'color': 'FF0000'}, 'ROI')

# Display the map 'm3'
m3

Map(center=[17.425718055675187, 78.47475850000585], controls=(WidgetControl(options=['position', 'transparent_…

In [5]:
# Calculate percentages
percentage_water = water_mask.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=roi,
    scale=10
).get('nd').getInfo() * 100

percentage_non_vegetation = non_vegetation_mask.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=roi,
    scale=10
).get('nd').getInfo() * 100

percentage_vegetation = vegetation_mask.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=roi,
    scale=10
).get('nd').getInfo() * 100

# Print the percentages
print(f'Percentage of Water: {percentage_water:.2f}%')
print(f'Percentage of Non-Vegetation: {percentage_non_vegetation:.2f}%')
print(f'Percentage of Vegetation: {percentage_vegetation:.2f}%')


Percentage of Water: 64.59%
Percentage of Non-Vegetation: 28.72%
Percentage of Vegetation: 52.48%
